# Loss Berechnen

In diesem Notebook bauen wir auf unserer statistischen Generierung auf, die wir uns in Vorlesung zwei erarbeitet haben.
Nun wollen wir aber einen Schritt weiter gehen - wir wollen uns objektiv berechnen, wie gut unserer Vorhersage tatsächlich ist.

Schritt 1: Wir nehmen den wichtigsten Teil des Codes aus der letzten Vorlesung.

Wir brauchen:
- Unsere Liste mit unseren Originaldaten (Trainingsdaten)
- Unsere Liste mit unseren Random generierten Namen (Baseline)
- Unsere Liste mit den statistisch generierten Namen

In [ ]:
%pip install torch

In [5]:
import torch

lines = open("data/vornamenstatistik_24.csv", "r", encoding="utf-8").read().splitlines()
names = list(set([n.split(",")[1] for n in lines][1:])) # Liste mit Originaldaten

sorted_names = sorted(names, key=lambda x: len(x))
count_chars = {}
for w in sorted_names:
    chars = ["<s>"] + list(w) + ["<e>"]
    for c1 in chars:
        count_chars[c1.lower()] = count_chars.get(c1.lower(), 0) + 1
all_chars = list(count_chars.keys())
sorted_chars = sorted(count_chars.items(), key=lambda x: x[1])
frequent_chars = [i for i in all_chars if count_chars[i] >= 10]

N_2 = torch.zeros((len(frequent_chars),len(frequent_chars)), dtype=torch.int32)

for w in sorted_names:
    chars = ["<S>"] + list(w) + ["<E>"]
    for c1, c2 in zip(chars, chars[1:]):
        if c1.lower() in frequent_chars and c2.lower() in frequent_chars:
            id_x = frequent_chars.index(c1.lower())
            id_y = frequent_chars.index(c2.lower())
        # Statt in unserem Dictionary hochzuzählen, zählen wir jetzt an der Matrixposition
        N_2[id_x, id_y] += 1

In [6]:
import random

random_names = [] # Liste mit Random Names als Baseline
for i in range(10):
    name = ""
    rand_len = random.randint(2,17)
    for i in range(rand_len):
        buchstabe = random.choice(frequent_chars)
        name += buchstabe
    random_names.append(name)
    
random_names

['yá',
 '<s>yeátmuéb<s>phsecy',
 'cmbnwopnfsnevqq',
 'kuná',
 'kc',
 'xgnxwreéaj--',
 'bj<e>degpk',
 'dtnl',
 'áz<s>x',
 'č<e>čéál<e>noeti<e>jd']

In [7]:
BT = torch.zeros(len(frequent_chars), len(frequent_chars))

for n in names:
    char = ["<s>"] + list(n) + ["<e>"]
    for c1, c2 in zip(char, char[1:]): 
        if c1.lower() in frequent_chars and c2.lower() in frequent_chars:
            id_x = frequent_chars.index(c1.lower())
            id_y = frequent_chars.index(c2.lower())
            BT[id_x, id_y] += 1

g = torch.Generator().manual_seed(42)

P = (BT).float() # Wir ersparen uns Zeit und Rechenleistung, in dem wir nicht jedes Mal die Float-konvertiereung machen müssen
P = P / P.sum(1, keepdims=True)

generierte_name = []  # Unsere Liste mit statistisch generierten Namen
for i in range(10):
    curr_char = "<s>"
    name = ""
    while curr_char != "<e>":
        p = P[frequent_chars.index(curr_char)]
        ind = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        char = frequent_chars[ind]
        curr_char = char
        name += curr_char
    generierte_name.append(name[:-3])
generierte_name

['lkaninri',
 'uakialavikonasasayualininannti',
 'n',
 'ma',
 'a',
 'iov',
 'keharima',
 'b',
 'wia',
 'sanel']

Schritt 2: Wir wollen jetzt berechnen, wie gut oder schlecht unser Modell uns Namen generiert. 
Da wir nur die statistische Häufigkeit von Bigrams bisher hinzugezogen haben, arbeiten wir mit dieser jetzt auch weiter. 

Wir schauen uns zunächst an, wie wahrscheinlich die Bigramme unserer Originaldaten sind.

In [8]:
for w in names[:3]:
    chars = ["<s>"] + list(w) + ["<e>"]
    for c1, c2 in zip(chars, chars[1:]):
        id_x = frequent_chars.index(c1.lower())
        id_y = frequent_chars.index(c2.lower())
        prob = P[id_x, id_y]
        print(f'{frequent_chars[id_x]}{frequent_chars[id_y]}: {prob}')

<s>m: 0.11987809091806412
ml: 0.0024906599428504705
la: 0.1507052779197693
ad: 0.02787456475198269
de: 0.16033755242824554
en: 0.13243408501148224
no: 0.0649966150522232
ov: 0.05088495463132858
v<e>: 0.0625
<s>m: 0.11987809091806412
ma: 0.4146949052810669
al: 0.07456445693969727
lv: 0.014847810380160809
vi: 0.3888888955116272
in: 0.15510204434394836
na: 0.1970209926366806
a<e>: 0.28710800409317017
<s>v: 0.02404334582388401
vo: 0.0243055559694767
ol: 0.10619468986988068
lk: 0.011135857552289963
ke: 0.10223641991615295
er: 0.0790925845503807
r<e>: 0.11347517371177673


Wir sehen - einige Kombinationen sind sehr häufig, wie zum Beispiel "ya", währen andere sehr selten sind, wie zum Beispiel "nz".

Um jetzt die Wahrscheinlichkeit eines Wortes zu berechnen, multipliziert man alle Einzelwahrscheinlichkeiten.

In [9]:
for w in names[:3]:
    likelihood = 1
    chars = ["<s>"] + list(w) + ["<e>"]
    for c1, c2 in zip(chars, chars[1:]):
        id_x = frequent_chars.index(c1.lower())
        id_y = frequent_chars.index(c2.lower())
        prob = P[id_x, id_y]
        likelihood *= prob
    print(f'{w}: {likelihood}')

Mladenov: 5.505363092489501e-12
Malvina: 1.877861848242901e-07
Volker: 6.341166480794413e-10


Von den ersten Namen hat "Amaya" den statistisch höchsten Wert. Aber die Werte werden alle sehr klein. Deswegen nehmen wir statt dem ursprünglichen Wert den Logarithmus. 

In [10]:
for w in names[:3]:
    log_likelihood = 0.0
    chars = ["<s>"] + list(w) + ["<e>"]
    for c1, c2 in zip(chars, chars[1:]):
        id_x = frequent_chars.index(c1.lower())
        id_y = frequent_chars.index(c2.lower())
        prob = P[id_x, id_y]
        logprob = torch.log(prob)
        log_likelihood += logprob
    print(f'{w}: {log_likelihood}')

Mladenov: -25.925296783447266
Malvina: -15.487961769104004
Volker: -21.178787231445312


Bei dieser Art von Berechnung ist der Wert besser, je näher er an der Null liegt. Leichter zu interpretieren wäre es, wenn wir daraus ein Minimierungsproblem machen könnten - also wenn kleine Werte => besser.
Dafür nehmen wir einfach die negative log-likelihood.

Eine letzte Modifikation bauen wir noch ein - statt die Werste einfach nur aufzusummieren und damit längere Namen zu benachteiligen, wollen wir den Mittelwert nehmen.

In [11]:
from statistics import mean

avg_log_likelihood = []
for w in names:
    log_likelihood = 0.0
    num_bigrams = 0
    chars = ["<s>"] + list(w) + ["<e>"]
    for c1, c2 in zip(chars, chars[1:]):
        if c1.lower() in frequent_chars and c2.lower() in frequent_chars:
            num_bigrams += 1
            id_x = frequent_chars.index(c1.lower())
            id_y = frequent_chars.index(c2.lower())
            prob = P[id_x, id_y]
            logprob = torch.log(prob)
            log_likelihood += logprob
    avg_log_likelihood.append((-log_likelihood / num_bigrams).item())
print(mean(avg_log_likelihood))

2.4544275189861757


Jetzt haben wir uns unsere Baseline berechnet - In unseren originalen Namensdaten haben Namen im Schnitt eine Negative Log-Likelihood von 2.45. 

In [18]:
avg_log_likelihood_rand = []
for w in random_names:
    log_likelihood = 0.0
    num_bigrams = 0
    chars = ["<s>"] + list(w) + ["<e>"]
    for c1, c2 in zip(chars, chars[1:]):
        if c1.lower() in frequent_chars and c2.lower() in frequent_chars:
            num_bigrams += 1
            id_x = frequent_chars.index(c1.lower())
            id_y = frequent_chars.index(c2.lower())
            prob = P[id_x, id_y]
            if prob == 0:
                prob = torch.tensor(0.00001)  # Für eine Wahrscheinlichkeit von 0 ist der logarithmus unendlich; hier nutzen wir eine leichte Art das zu verhindern
            logprob = torch.log(prob)
            log_likelihood += logprob
    avg_log_likelihood_rand.append((-log_likelihood / num_bigrams).item())
print(mean(avg_log_likelihood_rand))

6.791491985321045


Wir sehen - random generierte Namen haben eine deutliche schlechtere mittlere Wahrscheinlichkeit.

Aber wie sieht es jetzt mit unseren statistischn generierten Namen aus?

In [19]:
avg_log_likelihood_gen = []
for w in generierte_name:
    log_likelihood = 0.0
    num_bigrams = 0
    chars = ["<s>"] + list(w) + ["<e>"]
    for c1, c2 in zip(chars, chars[1:]):
        if c1.lower() in frequent_chars and c2.lower() in frequent_chars:
            num_bigrams += 1
            id_x = frequent_chars.index(c1.lower())
            id_y = frequent_chars.index(c2.lower())
            prob = P[id_x, id_y]
            if prob == 0:
                prob = torch.tensor(0.00001)  # Für eine Wahrscheinlichkeit von 0 ist der logarithmus unendlich; hier nutzen wir eine leichte Art das zu verhindern
            logprob = torch.log(prob)
            log_likelihood += logprob
    avg_log_likelihood_gen.append((-log_likelihood / num_bigrams).item())
print(mean(avg_log_likelihood_gen))

2.337066102027893


Statistisch stehen unsere generierten Namen sogar leicht besser da als unsere echten Vornamen. Auf jeden Fall erwarten wir einen Wert, der grob im gleichen Rahmen liegt wie unsere Ausgangsdaten: Darauf haben wir ja unsere statistische Berechnung trainiert, das heißt wir können auch kein Ergebnis erwarten, dass besser ist als unsere Trainingsdaten.